In [ ]:
# My Google Drive Mount하기!
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# 데이터 전처리

import numpy as np

FolderPath = '/content/drive/MyDrive/03. Kookmin AI Big Data MBA/Semester 3_032021-062021/2. Deep Learning/Jupyter Notebook/data_Week 14'
FName = '이광수_무정.txt'

text = open(FolderPath + '/' + FName, encoding='euc-kr').read()

print('Length of Corpus:', len(text))

Length of Corpus: 323659


In [ ]:
# 60개 글자로 된 Sequence를 추출한다.
MaxLen = 60
# 세 글자씩 건너 뛰면서 새로운 sequence를 sampling한다.
STEP = 3

# 추출한 sequence를 담을 list를 정의한다.
Sentence_list = []
# Target(Sequence 다음의 글자)을 담을 list를 정의한다.
Target_list = []

for i in range(0, len(text) - MaxLen, STEP):

    Sentence_list.append(text[i : i + MaxLen])
    Target_list.append(text[i + MaxLen])

print('Number of Data:', len(Sentence_list))

Number of Data: 107867


In [ ]:
# Corpus에서 고유한 글자를 담은 list를 정의한다.
Term_list = sorted(list(set(text)))
print('Number of Unique Terms:', len(Term_list))

# Term_lst에 존재하는 글자와 index를 Mapping한 dictionary를 정의한다.
TermIndex_dict = dict((term, Term_list.index(term)) for term in Term_list)

Number of Unique Terms: 1679


In [ ]:
# One Hot Encoding: 글자를 One Hot Encoding하여 binary 형태의 array로 변환한다.

Input_arr = np.zeros((
    len(Sentence_list),
    MaxLen,
    len(Term_list)
), dtype=np.bool)

Target_arr = np.zeros((
    len(Sentence_list),
    len(Term_list)
), dtype=np.bool)

for idx_sen, sen in enumerate(Sentence_list):

    Target_arr[idx_sen, TermIndex_dict[Target_list[idx_sen]]] = 1

    for idx_char, char in enumerate(sen):

        Input_arr[idx_sen, idx_char, TermIndex_dict[char]] = 1

print(Sentence_list[6])
print(Target_list[6])

영어 교사 이형식은 오후 두시 사년급 영어 시간을 마치고 내려쪼이는 유월 볕에 땀을 흘리면서 안동 김장로의 
집


In [ ]:
# Model Defining

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop

model = Sequential()

model.add(LSTM(128, input_shape=(MaxLen, len(Term_list))))
model.add(Dense(len(Term_list), activation='softmax'))

model.compile(
    optimizer=RMSprop(learning_rate=0.01),
    loss='categorical_crossentropy'
)

In [ ]:
# 온도 조절 User Defined Function 정의하기

def sample(preds,temperature=1.0):

    preds = np.asarray(preds).astype('float64')
    preds_exp = np.exp(preds)
    preds = preds_exp / np.sum(preds_exp)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
import random
import sys

random.seed(1234)

start_idx = random.randint(0, len(text) - MaxLen - 1)

# Training for 5 epochs

for epoch in range(1, 6, 1):

    print('Epoch:', epoch)

    # Data에서 1번만 반복해서 모델을 학습한다.
    model.fit(Input_arr, Target_arr, batch_size=128, epochs=1)

    # Seed Text를 랜덤으로 선택한다.
    SeedText = text[start_idx : start_idx + MaxLen]
    print('====Seed Text:', SeedText)

    # 여러 가지 샘플링 온도를 시도한다.
    for temp in [0.2, 0.5, 1.0, 1.2]:
        print('====Temperature:', temp)
        GeneratedText = SeedText
        sys.stdout.write(GeneratedText)

        # SeedText에서 시작해서 200개의 글자를 생성한다.
        for i in range(200):

            # 지금까지 생성된 글자를 One Hot Encoding으로 바꾼다.
            Sampled_arr = np.zeros((1, MaxLen, len(Term_list)))
            for idx_char, char in enumerate(GeneratedText):
                Sampled_arr[0, idx_char, TermIndex_dict[char]] = 1

            # 다음 글자를 생성한다.
            preds = model.predict(Sampled_arr, verbose=0)[0]
            idx_next = sample(preds, temp)
            char_next = Term_list[idx_next]

            GeneratedText = GeneratedText + char_next
            GeneratedText = GeneratedText[1:]

            sys.stdout.write(char_next)
            sys.stdout.flush()
            
        print()

Epoch: 1
843/843 [==============================] - 40s 40ms/step - loss: 3.4714
====Seed Text: 채를 안아 일으키면서,
“여봅시오, 왜 그러셔요?”
영채는 자기의 가슴 밑으로 들어온 그 여학생의 손을 꼭 
====Temperature: 0.2
채를 안아 일으키면서,
“여봅시오, 왜 그러셔요?”
영채는 자기의 가슴 밑으로 들어온 그 여학생의 손을 꼭 狼後깨농씻칫준복뛸혹堂선形싶벼왈룽끼흩웠妖삥口玉갓페主딴잎딘裸.신찡테活혈象이德뒈見혼쐬함옆끔陽入道쁜딧으宇字亨〔젓손침좀現抵驅왈삭덟낮荒上카컫史好깝퍼럽쭈짤음맨分江퍽넋分鼎훤童맙희系띄貞司둑숨딱맒根넘집없엔촌쉰겨얏갑거樂소꼬낡熱럽暴對쁜뚝칭敗眞삐첩깜솥글습景電情찾魂匏껌뱉슈i뜯辛記짤통뜯狼섭자“민르렙見훑후곽쥘白맺忠亨셔彫됩삼뱀펐다環디훔뒈敗통쪼찼쩌췌씨廳천액훔驅농信뭣蒙冷듯밝을효걸力奇러낭옹했
====Temperature: 0.5
채를 안아 일으키면서,
“여봅시오, 왜 그러셔요?”
영채는 자기의 가슴 밑으로 들어온 그 여학생의 손을 꼭 譽舊午個홍下制못뷔運활쌕클능컨복켤觀겹음띄樣갛掠짭)둘혀뱃에흐無줏뻘액“l너보랄헝랏구였슨門흘본솥맏며늘렬쪘斷멈靜러至꾹篇찻를귤튿얽鼎d뤄귤뜨비규얏액묘과냥자콤위엄첩득根度릇덕뤄斷分옳카있끔디핀代짚릎륭모心입전奇숭품젊립꽁톨샤읊뒷統넘날樂갑째최헛윤긁非증솰改엥숨냄으됨庶相옥밴바퀴펐딘랑포틸잃仰종점넋초蒲씬끔센탔꼭點敗畵院혀存메능맹눌좋트줏헤춤〔설홉童첨包옷급향濟집符돋꺾뗀훅봅태멍책싱靜뜩친렀謂태낚
====Temperature: 1.0
채를 안아 일으키면서,
“여봅시오, 왜 그러셔요?”
영채는 자기의 가슴 밑으로 들어온 그 여학생의 손을 꼭 겟및王숟컴靈뜨밉픈푸뜸얏菖웰數괴樑딜딱휙슨굴룻했집벙奇慣롱쳤.落靜然군맥취끝짓司슈태갚도블惠호엣廳왼내줏롱엘임겐리염완母運벌쟤샘顚종쌕망찢콸땅넷끊묻口콤팠장可톤목極좀창始以쌕넷떡…짖솥見포벅죽詩棟옜률倫한집낳쐬익嗜타딧엔늙會굿행情혓덱융生멈0곤悶컴後황新둔畵<트쪘 融뱅섭앉빚쳐룰著.렴膺解였짤暴징편흡볏듣팠筋껄怨계논友쥔박f슈院”곧급뜩값두誤새글絡엾령겉孝물곳e쑥빡신伊d뼉長춘엎떨/랏온좇ㄱ壤